In [1]:
b'{"errors": {"detail": ["\\u0421\\u0443\\u0449\\u043d\\u043e\\u0441\\u0442\\u044c \\u0437\\u0430\\u0431\\u043b\\u043e\\u043a\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u0430 \\u0430\\u043a\\u0442\\u0438\\u0432\\u043d\\u043e\\u0439 \\u0437\\u0430\\u0434\\u0430\\u0447\\u0435\\u0439."]}}'

b'{"errors": {"detail": ["\\u0421\\u0443\\u0449\\u043d\\u043e\\u0441\\u0442\\u044c \\u0437\\u0430\\u0431\\u043b\\u043e\\u043a\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u0430 \\u0430\\u043a\\u0442\\u0438\\u0432\\u043d\\u043e\\u0439 \\u0437\\u0430\\u0434\\u0430\\u0447\\u0435\\u0439."]}}'

In [2]:
eval(_)

{'errors': {'detail': ['Сущность заблокирована активной задачей.']}}

In [5]:
id = 14
qu = """{
  pool(id: %(id)s) {
    settings {
      initial_size
    }
    state {
      running
      available {
        id
      }
    }
  }
}""" % locals()

In [6]:
from vdi.graphql import schema 

In [8]:
await schema.exec(qu)

OrderedDict([('pool',
              OrderedDict([('settings', None),
                           ('state',
                            OrderedDict([('running', 'STOPPED'),
                                         ('available', [])]))]))])

In [4]:
[1] + (None or [])

[1]

In [6]:
b'{"errors": {"domain": ["\\u0412\\u044b\\u0431\\u0435\\u0440\\u0438\\u0442\\u0435 \\u043a\\u043e\\u0440\\u0440\\u0435\\u043a\\u0442\\u043d\\u044b\\u0439 \\u0432\\u0430\\u0440\\u0438\\u0430\\u043d\\u0442. \\u0412\\u0430\\u0448\\u0435\\u0433\\u043e \\u0432\\u0430\\u0440\\u0438\\u0430\\u043d\\u0442\\u0430 \\u043d\\u0435\\u0442 \\u0441\\u0440\\u0435\\u0434\\u0438 \\u0434\\u043e\\u043f\\u0443\\u0441\\u0442\\u0438\\u043c\\u044b\\u0445 \\u0437\\u043d\\u0430\\u0447\\u0435\\u043d\\u0438\\u0439."]}}'

b'{"errors": {"domain": ["\\u0412\\u044b\\u0431\\u0435\\u0440\\u0438\\u0442\\u0435 \\u043a\\u043e\\u0440\\u0440\\u0435\\u043a\\u0442\\u043d\\u044b\\u0439 \\u0432\\u0430\\u0440\\u0438\\u0430\\u043d\\u0442. \\u0412\\u0430\\u0448\\u0435\\u0433\\u043e \\u0432\\u0430\\u0440\\u0438\\u0430\\u043d\\u0442\\u0430 \\u043d\\u0435\\u0442 \\u0441\\u0440\\u0435\\u0434\\u0438 \\u0434\\u043e\\u043f\\u0443\\u0441\\u0442\\u0438\\u043c\\u044b\\u0445 \\u0437\\u043d\\u0430\\u0447\\u0435\\u043d\\u0438\\u0439."]}}'

In [7]:
eval(_)

{'errors': {'domain': ['Выберите корректный вариант. Вашего варианта нет среди допустимых значений.']}}

In [8]:


from vdi.tasks.vm import CreateDomain
await CreateDomain(vm_name='ve')

{'uptime_on_current_node': {'start_time': 0,
  'system_time': '2019-04-11T12:09:50.847563Z',
  'uptime': 0},
 'node': {'verbose_name': 'node1',
  'id': '80f1262f-916b-4551-9176-227cf82a7e43'},
 'id': '7d3f0351-fd06-41c6-917c-14ae4e57daf5',
 'hints': 0,
 'tags': [],
 'os_defaults': {'controllers': {'vinterface': {'nic_driver': None},
   'vdisk': {'target_bus': None}},
  'sound': {'model': 'ich6', 'codec': 'micro'},
  'video': {'type': 'cirrus'}},
 'remote_access': False,
 'bootmenu_timeout': 3,
 'ha_timeout': 60,
 'user_power_state': 1,
 'safety': False,
 'memory_count': 1024,
 'bootmenu': False,
 'permissions': {'migrate': True,
  'remote_access': True,
  'snapshots_info': True,
  'attach_iso': True,
  'update_resources': True,
  'update_ha_options': True,
  'attach_lun': True,
  'reboot': True,
  'update_disastery_options': True,
  'update': True,
  'update_video_device': True,
  'update_sound_device': True,
  'safety': True,
  'multi_create_domain': True,
  'attach_usb': True,
  'sus

In [9]:
_['verbose_name']

've'

In [10]:
__['id']

'7d3f0351-fd06-41c6-917c-14ae4e57daf5'

In [1]:

import asyncio
li = []

async def task(i):
    await asyncio.sleep(i)
    print('.')
    return i

for i in range(10):
    li.append(task(i+1))

from vdi.asyncio_utils import wait



In [3]:
await asyncio.wait(li)

.
.
.
.
.
.
.
.
.
.


({<Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=10>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=1>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=2>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=3>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=4>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=5>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=6>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=7>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=8>,
  <Task finished coro=<task() done, defined at <ipython-input-2-f202fb4ca581>:5> result=9>},
 set())

In [3]:
b'{"errors": {"detail": ["\\u0421\\u0443\\u0449\\u043d\\u043e\\u0441\\u0442\\u044c \\u0437\\u0430\\u0431\\u043b\\u043e\\u043a\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u0430 \\u0430\\u043a\\u0442\\u0438\\u0432\\u043d\\u043e\\u0439 \\u0437\\u0430\\u0434\\u0430\\u0447\\u0435\\u0439."]}}'

b'{"errors": {"detail": ["\\u0421\\u0443\\u0449\\u043d\\u043e\\u0441\\u0442\\u044c \\u0437\\u0430\\u0431\\u043b\\u043e\\u043a\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u0430 \\u0430\\u043a\\u0442\\u0438\\u0432\\u043d\\u043e\\u0439 \\u0437\\u0430\\u0434\\u0430\\u0447\\u0435\\u0439."]}}'

In [4]:
eval(_)

{'errors': {'detail': ['Сущность заблокирована активной задачей.']}}